# 0.0. Imports

In [1]:
# pip install pandas seaborn numpy matplotlib

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

import pickle
import os


## 0.1. Helper Functions

In [3]:
def change_date(data, column):
    data[column] = pd.to_datetime(data[column], format='%Y-%m-%d')
    return data[column]

def descrever(df):
    num_attributes = df.select_dtypes(['int64', 'float64'])

    mean = pd.DataFrame(num_attributes.apply(np.mean)).T
    median = pd.DataFrame(num_attributes.apply(np.median)).T

    min_ = pd.DataFrame(num_attributes.apply(np.min)).T
    max_ = pd.DataFrame(num_attributes.apply(np.max)).T
    std = pd.DataFrame(num_attributes.apply(np.std)).T
    range_ =  pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
    skew = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
    kurtosis  = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T
    unique  = pd.DataFrame(num_attributes.apply(lambda x: x.nunique())).T


    metricas = pd.concat([ min_, max_, range_, mean, median, std, skew, kurtosis, unique]).T
    metricas.columns = [ 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis', 'unique']
    return metricas

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)

## 0.2. Load original datasets

In [18]:
df_geral = pd.read_csv('../data/portfolio_geral.csv', encoding='UTF-8')
df_cliente = pd.read_csv('../data/portfolio_clientes.csv', encoding='UTF-8')
df_comunicados = pd.read_csv('../data/portfolio_comunicados.csv', encoding='UTF-8')
df_tpv = pd.read_csv('../data/portfolio_tpv.csv', encoding='UTF-8')


# 1.0. Mudança de tipos

In [19]:
#mudança de tipos

#dates
dates = ['dt_contrato', 'dt_desembolso','dt_vencimento', 'dt_wo', 'dt_ref_portfolio' ]
for date in dates:
    df_geral[date] = change_date(df_geral, date)

df_comunicados['data_acao'] = change_date(df_comunicados, 'data_acao')
df_comunicados['dt_ref_portfolio'] = change_date(df_comunicados, 'dt_ref_portfolio')

df_tpv['dt_transacao'] = pd.to_datetime(df_tpv['dt_transacao'], format='%Y%m%d')




# 2.0. Tratamentos e Joins

## 2.1. Split portfolio geral

In [20]:
#split geral -> geral e contrato
df_contrato = df_geral[['contrato_id', 'nr_documento', 'safra', 'dt_contrato', 'dt_desembolso',
                        'dt_vencimento','dt_wo', 'prazo', 'vlr_desembolsado',
                        'vlr_tarifa', 'juros_mes','juros_diario']].drop_duplicates('contrato_id')

df_geral = df_geral.drop(['safra', 'dt_contrato', 'dt_desembolso',
                        'dt_vencimento','dt_wo', 'prazo', 'vlr_desembolsado',
                        'vlr_tarifa', 'juros_mes','juros_diario'], axis=1)

## 2.2. Filtrar linhas do df_geral

In [21]:
#filtrar geral -> manter somente primeira linha de settled

#sort df_geral por id e date
df_geral = df_geral.sort_values(['contrato_id', 'dt_ref_portfolio']).reset_index(drop=True)

#pega todas linhas com status quitado
aux = df_geral[df_geral['status_contrato']=='Settled']

#pega data minima de cada contrato quitado
aux0 = aux[['contrato_id', 'dt_ref_portfolio']].groupby('contrato_id').min().reset_index()

#juntar outras features 
aux0 = pd.merge(aux0, df_geral, on=['contrato_id', 'dt_ref_portfolio'], how='inner')

#remover todos status quitados de df_geral
df_geral = df_geral[df_geral['status_contrato']!='Settled']

#colocar de volta apenas o primeiro registro do contrato quitado
df_geral = pd.concat([df_geral, aux0])



In [22]:
#encontrar dados estranhos
descrever(df_geral)

,min,max,range,mean,median,std,skew,kurtosis,unique
perc_retencao,0.0114,1.20,1.1886,0.207289,0.156,0.242711,3.425979,11.127067,2301.0
vlr_pgto_realizado,-3195.5600,210216.25,213411.8100,86.328751,0.000,452.963586,127.412612,35987.554675,111177.0
vlr_pgto_esperado,0.5700,1134.03,1133.4600,112.022508,70.700,124.339717,2.586350,8.586397,12604.0
vlr_saldo_devedor,-44613.3700,750102.75,794716.1200,29216.660267,15216.420,41036.658143,3.663851,21.157096,2878673.0
vlr_saldo_devedor_esperado,0.0000,401338.69,401338.6900,19754.802032,9216.805,30367.406702,3.450363,16.585532,2948472.0
dsp,0.0000,791.00,791.0000,21.725396,1.000,62.933864,4.474875,24.865537,792.0
dspp,0.0000,833.00,833.0000,58.527839,3.000,114.092322,2.513723,6.531199,834.0
flag_transacao,0.0000,1.00,1.0000,0.568374,1.000,0.495303,-0.276088,-1.923776,2.0


In [23]:
#limpar dados estranhos

# vlr_pgto_realizado<0
linhas = len(df_geral[df_geral['vlr_pgto_realizado']<0])
print(f'Linhas removidas para valores de pagamento negativo: {linhas}')
df_geral = df_geral[df_geral['vlr_pgto_realizado']>=0]

# vlr_saldo_devedor <0
linhas = len(df_geral[df_geral['vlr_saldo_devedor']<0])
print(f'Linhas removidas para valores de pagamento negativo: {linhas}')
df_geral = df_geral[df_geral['vlr_saldo_devedor']>=0]


# df_geral[df_geral['vlr_saldo_devedor']<0]

Linhas removidas para valores de pagamento negativo: 23
Linhas removidas para valores de pagamento negativo: 1591


## 2.3. Join -  portifólio geral e de comunicados

In [24]:
#merge geral e comunicados
df_geral_comunicado = pd.merge(df_geral, df_comunicados, on=['contrato_id', 'dt_ref_portfolio'], how='left')


## 2.4. Join - portfólio geral e de tpv

In [25]:
#encontrar dados estranhos tpv
descrever(df_tpv)

,min,max,range,mean,median,std,skew,kurtosis,unique
qtd_transacoes,0.0,1245.00,1245.00,15.910281,7.0,26.155925,6.003473,70.741882,703.0
vlr_tpv,0.0,176880.93,176880.93,888.487945,430.0,1657.037450,11.062951,341.617044,491200.0


In [26]:
#limpar dados estranhos
linhas = len(df_tpv[df_tpv['qtd_transacoes']<0])
print(f'Linhas removidas para valores de quantidade de transação negativo: {linhas}')
df_tpv = df_tpv[df_tpv['qtd_transacoes']>=0]

linhas = len(df_tpv[df_tpv['vlr_tpv']<0])
print(f'Linhas removidas para valores de transação de transação negativo: {linhas}')
df_tpv = df_tpv[df_tpv['vlr_tpv']>=0]


Linhas removidas para valores de quantidade de transação negativo: 0
Linhas removidas para valores de transação de transação negativo: 0


In [27]:
#merge geral e tpv
df_geral = pd.merge(df_geral, df_tpv.rename(columns = {'dt_transacao':'dt_ref_portfolio'}), on=['nr_documento', 'dt_ref_portfolio'], how='left')
df_geral = df_geral.fillna(0)

## 2.5. Join - df_contrato e portfólio de clientes


In [28]:
#merge contratos e clientes
df_contrato = pd.merge(df_contrato, df_cliente, on='nr_documento', how='inner')

# 3.0. Feature Engineering

### 3.1. Features em df_geral

In [29]:
#valor debitado desconsiderando pagamento 
df_geral['debito_sem_pagamento'] = df_geral['vlr_saldo_devedor'] + df_geral['vlr_pgto_realizado']


#valor pago tpv = valor * perc_retencao/100
df_geral['valor_pago_tpv'] = df_geral['vlr_tpv'] * df_geral['perc_retencao']/100


### 3.2. Features em df_contrato

In [30]:
#extrair ultima data de cada contrato
last_date = df_geral[['contrato_id', 'dt_ref_portfolio']].groupby('contrato_id').max().reset_index()
last_date = pd.merge(last_date, df_geral[['contrato_id', 'dt_ref_portfolio','status_contrato']], on=['contrato_id', 'dt_ref_portfolio'], how='inner')

#conseguir o ultimo status de contrato e ultima data
df_contrato = pd.merge(df_contrato, last_date, on='contrato_id', how='inner').rename(columns={'dt_ref_portfolio':'ultima_data'})

#mean dsp / contrato
aux = df_geral[['dsp', 'contrato_id']].groupby('contrato_id').mean().reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#mean dspp / contrato
aux = df_geral[['dspp', 'contrato_id']].groupby('contrato_id').mean().reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#sum vlr pago realizado / contrato
aux = df_geral[['vlr_pgto_realizado', 'contrato_id']].groupby('contrato_id').sum().reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#sum vlr pago tpv realizado / contrato
aux = df_geral[['valor_pago_tpv', 'contrato_id']].groupby('contrato_id').sum().reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#valor total pago / contrato
df_contrato['valor_total_pago'] = df_contrato['vlr_pgto_realizado'] + df_contrato['valor_pago_tpv']

#valor final que deveria ser pago desconsiderando pagamento
aux = df_geral[['debito_sem_pagamento','contrato_id']].groupby('contrato_id').max().reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#dias esperado de contrato
df_contrato['tempo_esperado'] = (df_contrato['dt_vencimento'] - df_contrato['dt_desembolso']).dt.days

#dias efetivos de contrato 
aux = df_geral[['contrato_id', 'dt_ref_portfolio']].groupby('contrato_id').count().rename(columns={'dt_ref_portfolio': 'dias_de_contrato'}).reset_index()
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='left')

#acoes totais
aux = df_geral_comunicado[['contrato_id', 'status']].groupby('contrato_id').count().reset_index().rename(columns={'status': 'n_acoes_total'})
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='inner')

#acoes entregues
aux = df_geral_comunicado [df_geral_comunicado['status']!='NAO ENTREGUE'] [['contrato_id', 'status']].groupby('contrato_id').count().reset_index().rename(columns={'status': 'n_acoes_entregues'})
df_contrato = pd.merge(df_contrato, aux, on='contrato_id', how='inner')


In [31]:
#check duplicates

#pegando um exemplo para analisar
index = df_contrato[df_contrato['contrato_id'].duplicated()==True].index[1]
id_ = df_contrato.iloc[192,0]
df_contrato[df_contrato['contrato_id']==id_]

,contrato_id,nr_documento,safra,dt_contrato,dt_desembolso,dt_vencimento,dt_wo,prazo,vlr_desembolsado,vlr_tarifa,juros_mes,juros_diario,tipo_empresa,cidade,estado,subsegmento,segmento,ultima_data,status_contrato,dsp,dspp,vlr_pgto_realizado,valor_pago_tpv,valor_total_pago,debito_sem_pagamento,tempo_esperado,dias_de_contrato,n_acoes_total,n_acoes_entregues
191,712b52a60e6e423d1540d197da718844,3316a7fc9281319e4d756a437c36bb9a,2021-01,2021-01-23,2021-01-26,2022-04-23,2023-04-23,25.65,14837.67,0.0,0.04788,0.00157,PJ,Rio de Janeiro,RJ,Outros,Bens duráveis,2022-04-18,Active,32.997768,46.399554,10879.8,51.13206,10930.93206,14931.82,452,448,26,9
192,712b52a60e6e423d1540d197da718844,3316a7fc9281319e4d756a437c36bb9a,2021-01,2021-01-23,2021-01-26,2022-04-23,2023-04-23,25.65,14837.67,0.0,0.04788,0.00157,PF,Rio de Janeiro,RJ,Lojas Diversas,Varejo,2022-04-18,Active,32.997768,46.399554,10879.8,51.13206,10930.93206,14931.82,452,448,26,9


A diferença está no tipo de empresa, subsegmento, segmento

In [32]:
#check numero de duplicados
duplicados = sum(df_contrato['contrato_id'].duplicated()==True)
print(f'Número de duplicados {duplicados}\n')

print('Em relação ao total: {:.3f}%'.format(duplicados/len(df_contrato)*100))

Número de duplicados 144

Em relação ao total: 0.966%


Não haverá grande perda de informação, portanto, os duplicados serão removidos.

In [37]:
#remove duplicates
df_contrato = df_contrato.drop_duplicates('contrato_id')

# 4.0. Criação de tabelas

## 4.1. Criação da df_mensagem

Um dataframe chamado df_mensagem foi criado apenas com as linhas do df_geral que continha uma comunicação com status diferente de “NAO ENTREGUE” e os 5 registros seguintes. O objetivo desse dataframe é observar o efeito da comunicação nos 5 dias seguintes, através do dsp e dspp. Dessa vez o enfoque é descobrir a curva ideal de vezes que se deve acionar o cliente.


In [38]:
# #pegar linha da ação
df_sorted = df_geral_comunicado.sort_values(['contrato_id', 'dt_ref_portfolio']).reset_index(drop=True)

#get index of not null messages
index_mensagem = df_sorted.loc[~df_sorted['status'].isna(), :].index

index_mensagem_expanded = index_mensagem
#create index list expanded

index_aux = index_mensagem + 1
index_mensagem_expanded = index_mensagem_expanded.append(index_aux)
index_aux+=1
index_mensagem_expanded = index_mensagem_expanded.append(index_aux)
index_aux+=1
index_mensagem_expanded = index_mensagem_expanded.append(index_aux)
index_aux+=1
index_mensagem_expanded = index_mensagem_expanded.append(index_aux)
index_aux+=1
index_mensagem_expanded = index_mensagem_expanded.append(index_aux)


#get df mensagem
df_mensagem = df_sorted.iloc[index_mensagem_expanded, :].sort_values(['contrato_id', 'dt_ref_portfolio']).drop_duplicates().reset_index(drop=True)

#get index das mensagens 
index_mensagem = df_mensagem.loc[df_mensagem['status'].isin(['LIDO', 'RESPONDIDO']), :].index

#create eficiencia
for index, i in enumerate(index_mensagem):
#     print(f'{index}/{len(index_mensagem)}')
    
    if df_mensagem.loc[i,'status']: #se status existe

        if df_mensagem.loc[i,'dsp']==0:   #se dsp ==0, passa
            pass

        else:                               #se dsp!=0, verifica os proximos 5 se vai ser igual a 0
            for i2 in range(i+1, i+6):
                if df_mensagem.loc[i2, 'dsp']==0:

                    df_mensagem.loc[i,'eficiencia']=1
    
                    if df_mensagem.loc[i2, 'dspp']==0:
                        df_mensagem.loc[i,'eficiencia']=2
                        
                else: 
                    pass
        

#preencher valores nulos de eficiencia em mensagens lidas com 0
df_mensagem['eficiencia'] = df_mensagem[df_mensagem['status'].isin(['LIDO','RESPONDIDO'])]['eficiencia'].fillna(0)

df_mensagem = df_mensagem[~df_mensagem['status'].isna()]

# 5.0. Exportar para csv

In [47]:
df_contrato.to_csv('../data/new_dfs/df_contrato.csv', index=False)
df_geral.to_csv('../data/new_dfs/df_geral.csv', index=False)
df_geral_comunicado.to_csv('../data/new_dfs/df_geral_comunicado.csv', index=False)
df_mensagem.to_csv('../data/new_dfs/df_mensagem.csv', index=False)

# 6.0. DataFrame previews

## 6.1. Df_Contrato


In [43]:
df_contrato.head()

,contrato_id,nr_documento,safra,dt_contrato,dt_desembolso,dt_vencimento,dt_wo,prazo,vlr_desembolsado,vlr_tarifa,juros_mes,juros_diario,tipo_empresa,cidade,estado,subsegmento,segmento,ultima_data,status_contrato,dsp,dspp,vlr_pgto_realizado,valor_pago_tpv,valor_total_pago,debito_sem_pagamento,tempo_esperado,dias_de_contrato,n_acoes_total,n_acoes_entregues
0,356c02706c8e74b15004bb5964ade6bb,b0871d4e0d72afd1d44381e5b2453cb2,2020-06,2020-06-24,2020-06-29,2020-12-24,2021-12-24,10.26,22686.57,0.0,0.10788,0.00345,PF,João Pessoa,PB,Educação,Serviços recorrentes,2020-08-21,Settled,0.722222,0.722222,24673.98,206.492544,24880.472544,22751.75,178,54,0,0
1,a5940634cbbe2a6d01d0676f6dfd6368,b0871d4e0d72afd1d44381e5b2453cb2,2020-08,2020-08-24,2020-08-26,2022-02-24,2023-02-24,30.78,59018.94,0.0,0.09588,0.00308,PF,João Pessoa,PB,Educação,Serviços recorrentes,2022-04-18,Active,34.474210,74.061564,58600.16,471.752528,59071.912528,71562.83,547,601,58,30
2,60a0f8d19ec695ea4f79710212cfd6dd,b0871d4e0d72afd1d44381e5b2453cb2,2021-03,2021-03-02,2021-03-04,2022-07-02,2023-07-02,27.36,25752.60,0.0,0.09588,0.00308,PF,João Pessoa,PB,Educação,Serviços recorrentes,2022-04-18,Active,56.795620,64.364964,12761.39,70.663846,12832.053846,39264.01,485,411,70,41
3,1eb0e13f83f336146227ccc9efb08c47,938513461b4a4fcd2bf99ffd8f12b9be,2020-07,2020-07-26,2020-07-28,2021-10-26,2022-10-26,25.65,16074.00,0.0,0.04788,0.00157,MEI,Campo Grande,MS,Alimentação Rápida,Alimentação,2021-06-08,Settled,0.598101,0.727848,19954.35,144.779233,20099.129233,16113.74,455,316,0,0
4,8b03675f597974ca96f06648e96af6d4,4303bd71ee3fe80aba4efc8dbd6c7d70,2020-08,2020-08-13,2020-08-17,2022-02-13,2023-02-13,30.78,17100.00,0.0,0.05388,0.00176,PJ,Mogi Mirim,SP,Outros,Serviços,2021-12-07,Settled,0.387029,0.414226,22360.45,138.366851,22498.816851,17125.05,545,478,0,0


## 6.2. Df_geral

In [44]:
df_geral.head()

,contrato_id,dt_ref_portfolio,nr_documento,status_contrato,perc_retencao,vlr_pgto_realizado,vlr_pgto_esperado,vlr_saldo_devedor,vlr_saldo_devedor_esperado,dsp,dspp,flag_transacao,qtd_transacoes,vlr_tpv,debito_sem_pagamento,valor_pago_tpv
0,000180509391a5ac66ff83cae603ffb8,2020-06-15,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,0.00,27.45,6932.34,6924.81,0,0,0,0.0,0.0,6932.34,0.00000
1,000180509391a5ac66ff83cae603ffb8,2020-06-16,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,0.00,27.45,6952.26,6917.27,1,1,1,6.0,191.5,6952.26,0.18384
2,000180509391a5ac66ff83cae603ffb8,2020-06-17,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,25.43,27.45,6946.81,6909.70,0,2,1,11.0,311.5,6972.24,0.29904
3,000180509391a5ac66ff83cae603ffb8,2020-06-18,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,41.40,27.45,6925.38,6902.11,0,0,1,16.0,464.0,6966.78,0.44544
4,000180509391a5ac66ff83cae603ffb8,2020-06-19,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,61.49,27.45,6883.79,6894.50,0,0,1,18.0,595.0,6945.28,0.57120


## 6.3. Df_geral_comunicado

In [45]:
df_geral_comunicado.head()

,contrato_id,dt_ref_portfolio,nr_documento,status_contrato,perc_retencao,vlr_pgto_realizado,vlr_pgto_esperado,vlr_saldo_devedor,vlr_saldo_devedor_esperado,dsp,dspp,flag_transacao,data_acao,tipo_acao,acao,status
0,000180509391a5ac66ff83cae603ffb8,2020-06-15,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,0.00,27.45,6932.34,6924.81,0,0,0,NaT,NaN,NaN,NaN
1,000180509391a5ac66ff83cae603ffb8,2020-06-16,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,0.00,27.45,6952.26,6917.27,1,1,1,NaT,NaN,NaN,NaN
2,000180509391a5ac66ff83cae603ffb8,2020-06-17,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,25.43,27.45,6946.81,6909.70,0,2,1,NaT,NaN,NaN,NaN
3,000180509391a5ac66ff83cae603ffb8,2020-06-18,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,41.40,27.45,6925.38,6902.11,0,0,1,NaT,NaN,NaN,NaN
4,000180509391a5ac66ff83cae603ffb8,2020-06-19,7996daab1bbe000bb5d1cc1bf317f390,Active,0.096,61.49,27.45,6883.79,6894.50,0,0,1,NaT,NaN,NaN,NaN


## 6.4. Df_mensagem

In [46]:
df_mensagem.head()


,contrato_id,dt_ref_portfolio,nr_documento,status_contrato,perc_retencao,vlr_pgto_realizado,vlr_pgto_esperado,vlr_saldo_devedor,vlr_saldo_devedor_esperado,dsp,dspp,flag_transacao,data_acao,tipo_acao,acao,status,eficiencia
0,000180509391a5ac66ff83cae603ffb8,2020-12-29,7996daab1bbe000bb5d1cc1bf317f390,Active,0.06,0.0,27.45,1960.10,4929.02,5,5,0,2020-12-29,EMAIL,campanhaobservacao,LIDO,0.0
1,000180509391a5ac66ff83cae603ffb8,2020-12-29,7996daab1bbe000bb5d1cc1bf317f390,Active,0.06,0.0,27.45,1960.10,4929.02,5,5,0,2020-12-29,HSM,campanhaobservacao,NAO ENTREGUE,NaN
6,000180509391a5ac66ff83cae603ffb8,2021-01-03,7996daab1bbe000bb5d1cc1bf317f390,Active,0.06,0.0,27.45,1988.44,4862.23,10,10,0,2021-01-05,EMAIL,campanhaparcelamento,ENTREGUE,NaN
7,000180509391a5ac66ff83cae603ffb8,2021-01-03,7996daab1bbe000bb5d1cc1bf317f390,Active,0.06,0.0,27.45,1988.44,4862.23,10,10,0,2021-01-05,HSM,campanhaparcelamento,ENTREGUE,NaN
13,000c35a61297edadc2842f6d5b4028e1,2020-10-28,1191ebfa94d3ca2e8a02f696aafde4a4,Active,0.18,0.0,106.19,33300.42,32474.02,5,9,0,2020-10-28,EMAIL,campanhaobservacao,ENTREGUE,NaN
